In [ ]:
# nbdev: hide this cell and assign exported code to `model_analysis` by default
# hide
# default_exp model_analysis

from nbdev.showdoc import *

# Visualizing Model States
> quick exploratory analysis of model states at arbitrary points along simulation

We often simulate a simple free recall experiment and visualize model states throughout to explore their capacity to
exhibit classical patterns of primacy, recency, and temporal contiguity. Any arbitrary configuration of parameters can
be specified for the model, including an `experiment_count`, determining the number of simulations with the given
parameters.

In each experiment:
1. A specified number of unique items are each experienced once,
2. Context is momentarily drifted toward its pre-experimental state, and
3. The model freely recalls items until it stops, with retrieval of previously experienced items disallowed.

To visualize model state, we add to our `model_analysis` submodule three basic categories of visualizations. To
visualize model state throughout encoding, we track the state of `context` and the amount of `support` for recall of
each item based on contextual state. We also prepare a visualization of the final state of `memory` once encoding is
finished. To visualize model state throughout retrieval, we similarly track `context` and `support` at each step of
recall. An additional visualization makes clearer the distribution of outcome probabilities at a particular index of
recall (e.g. after a second item has been recalled).

## Demo Parameters
For a demo of these functions, you can specify default model parameters (or even alter which model is considered) in the following code cell:

In [ ]:
# you can change the imported model and assign it the alias CMR and use it for 
# this demo if it supports the same functions
from repfr.models import DefaultCMR as CMR

# entries and values in this dict will be passed as model parameters
cmr_parameters = {
}

## Plotting Encoding States
First we create simulations and visualizations to track model state throughout encoding of new memories. 

To do this,we produce two parallel functions, `encoding_states` and `plot_states` that collect and visualize encoding states, respectively. 

An additional wrapper function called `encoding_visualizations` plots these states in addition to the
final overall state of model memory.

In [ ]:
#export

import numpy as np

def encoding_states(model):
    """
    Tracks state of context, and item supports across encoding. Model is also advanced to a state of fully encoded
    memories.

    **Required model attributes**:
    - item_count: specifies number of items encoded into memory
    - context: vector representing an internal contextual state
    - experience: adding a new trace to the memory model
    - activations: function returning item activations given a vector probe
    - outcome_probabilities: function returning item supports given a set of activations

    **Returns** array representations of context and support for retrieval of each item at each increment of item
    encoding. Each has shape model.item_count by model.item_count + 1.
    """

    experiences = np.eye(model.item_count, model.item_count + 1, 1)
    cmr_experiences = np.eye(model.item_count, model.item_count)
    encoding_contexts, encoding_supports = model.context, []

    # track model state across experiences
    for i in range(len(experiences)):
        model.experience(cmr_experiences[i].reshape((1, -1)))

        # track model contexts and item supports
        encoding_contexts = np.vstack((encoding_contexts, model.context))

        activation_cue = lambda model: model.context

        if len(encoding_supports) > 0:
            encoding_supports = np.vstack((encoding_supports, model.outcome_probabilities(activation_cue(model))))
        else:
            encoding_supports = model.outcome_probabilities(activation_cue(model))

    return encoding_contexts, encoding_supports

In [ ]:
# export

import seaborn as sns
import matplotlib.pyplot as plt

def plot_states(matrix, title, figsize=(15, 15), savefig=False):
    """
    Plots an array of model states as a value-annotated heatmap with an arbitrary title.

    **Arguments**:
    - matrix: an array of model states, ideally with columns representing unique feature indices and rows
        representing unique update indices
    - title: a title for the generated plot, ideally conveying what array values represent at each entry
    - savefig: boolean deciding whether generated figure is saved (True if Yes)
    """
    
    plt.figure(figsize=figsize)
    sns.heatmap(matrix, annot=True, linewidths=.5)
    plt.title(title)
    plt.xlabel('Feature Index')
    plt.ylabel('Update Index')
    if savefig:
        plt.savefig('figures/{}.jpeg'.format(title).replace(' ', '_').lower(), bbox_inches='tight')
    plt.show()

In [ ]:
# export

def encoding_visualizations(model, savefig=True):
    """
    Plots encoding contexts, encoding supports as heatmaps.

    **Required model attributes**:
    - item_count: specifies number of items encoded into memory
    - context: vector representing an internal contextual state
    - experience: adding a new trace to the memory model
    - activations: function returning item activations given a vector probe
    - outcome_probabilities: function returning item supports given a set of activations
    - memory: a unitary representation of the current state of memory

    **Also** requires savefig:  boolean deciding if generated figure is saved
    """

    encoding_contexts, encoding_supports = encoding_states(model)
    plot_states(encoding_contexts, 'Encoding Contexts', savefig=savefig)
    plot_states(encoding_supports, 'Supports For Each Item At Each Increment of Encoding', savefig=savefig)

### Demo

In [ ]:
model = CMR(**cmr_parameters)
encoding_visualizations(model)

`plot_states` is flexible enough to visualize other model representations, too:

In [ ]:
model = CMR(**parameters)
encoding_states(model)
print(model.__class__.__name__)
plot_states(model.mfc, 'CMR Mfc')
plot_states(model.mcf, 'CMR Mcf')

## Plotting Retrieval States

Tracking model state across each step of retrieval. Since retrieval stochastic, these values change with each random seed. An additional optional parameter `first_recall_item` can control which item is recalled first by the model (`0` denotes termination of recall while actual items are 1-indexed); it is useful for testing hypotheses about model dynamics during recall. We leave the parameter set at `None`, for now, indicating no controlled first recall.

In [ ]:
# export

import numpy as np

def retrieval_states(model, first_recall_item=None):
    """
    Tracks state of context, and item supports across retrieval. Model is also advanced into a state of
    completed free recall.

    **Required model attributes**:
    - item_count: specifies number of items encoded into memory
    - context: vector representing an internal contextual state
    - experience: adding a new trace to the memory model
    - activations: function returning item activations given a vector probe
    - outcome_probabilities: function returning item supports given a set of activations
    - free_recall: function that freely recalls a given number of items or until recall stops
    - state: indicates whether model is encoding or engaged in recall with a string

    **Also** optionally uses first_recall_item: can specify an item for first recall

    **Returns** array representations of context and support for retrieval of each item at each increment of item
    retrieval. Also returns recall train associated with simulation.
    """

    activation_cue = lambda model: model.context

    # encoding items, presuming model is freshly initialized
    encoding_states(model)
    retrieval_contexts, retrieval_supports = model.context, model.outcome_probabilities(
        activation_cue(model))

    # pre-retrieval distraction
    model.free_recall(0)
    retrieval_contexts = np.vstack((retrieval_contexts, model.context))
    retrieval_supports = np.vstack((retrieval_supports, model.outcome_probabilities(
        activation_cue(model))))

    # optional forced first item recall
    if first_recall_item is not None:
        model.force_recall(first_recall_item)
        retrieval_contexts = np.vstack((retrieval_contexts, model.context))
        retrieval_supports = np.vstack((retrieval_supports, model.outcome_probabilities(
            activation_cue(model))))

    # actual recall
    while model.retrieving:
        model.free_recall(1)
        retrieval_contexts = np.vstack((retrieval_contexts, model.context))
        retrieval_supports = np.vstack((retrieval_supports, model.outcome_probabilities(
            activation_cue(model))))

    return retrieval_contexts, retrieval_supports, model.recall[:model.recall_total]

In [ ]:
# export

def outcome_probs_at_index(model, support_index_to_plot=1, savefig=True):
    """
    Plots outcome probability distribution at a specific index of free recall.

    **Required model attributes**:
    - item_count: specifies number of items encoded into memory
    - context: vector representing an internal contextual state
    - experience: adding a new trace to the memory model
    - activations: function returning item activations given a vector probe
    - outcome_probabilities: function returning item supports given a set of activations
    - free_recall: function that freely recalls a given number of items or until recall stops
    - state: indicates whether model is encoding or engaged in recall with a string

    **Other arguments**:
    - support_index_to_plot: index of retrieval to plot
    - savefig: whether to save or display the figure of interest

    **Generates** a plot of outcome probabilities as a line graph. Also returns vector representation of the
    generated probabilities.
    """

    retrieval_supports = retrieval_states(model)[1]
    plt.plot(np.arange(model.item_count + 1), retrieval_supports[support_index_to_plot])
    plt.xlabel('Choice Index')
    plt.ylabel('Outcome Probability')
    plt.title('Outcome Probabilities At Recall Index {}'.format(support_index_to_plot))
    plt.show()
    return retrieval_supports[support_index_to_plot]

In [ ]:
# export

def retrieval_visualizations(model, savefig=True):
    """
    Plots incremental retrieval contexts and supports, as heatmaps, and prints recalled items.

    **Required model attributes**:
    - item_count: specifies number of items encoded into memory
    - context: vector representing an internal contextual state
    - experience: adding a new trace to the memory model
    - activations: function returning item activations given a vector probe
    - outcome_probabilities: function returning item supports given a set of activations

    **Also** uses savefig: boolean deciding whether figures are saved (True) or displayed
    """

    retrieval_contexts, retrieval_supports, recall = retrieval_states(model)
    plot_states(retrieval_contexts, 'Retrieval Contexts', savefig=savefig)
    plot_states(retrieval_supports, 'Supports For Each Item At Each Increment of Retrieval',
                savefig=savefig)
    return recall

### Demo

In [ ]:
model = CMR(**cmr_parameters)
retrieval_visualizations(model)